# xnatpy: a pythonic feeling interface to XNAT

xnatpy attempts to expose objects in XNAT as native feeling Python objects. The objects reflect the state of XNAT and changes to the objects automatically update the server.

To facilitate this xnatpy scans the server xnat.xsd and creates a Python class structure to mimic this is well as possible.

Current features:
* automatic generate of most data structures from the xnat.xsd
* easy exploration of data
* easy getting/setting of custom variables
* easy downloading/uploading of data
* using the prearchive
* the import service

Missing features (aka my TODO list):
* good support for the creation of objects
* good support for searches

### Some imports and helper code used later on

In [1]:
import os
import random

## getting started

First we need to set up an xnatpy session. 
The session scans the xnat.xsd, creates classes,
logs in into XNAT, and keeps the connection alive
using a hearbeat.

In [2]:
import xnat
session = xnat.connect('https://central.xnat.org', user='nosetests', password='nose2016')

[INFO] Retrieving schema from https://central.xnat.org/schemas/xnat/xnat.xsd


To save your login you set up a .netrc file with the correct information about the target host.
A simple example of a .netrc file can be found at  [http://www.mavetju.org/unix/netrc.php](http://www.mavetju.org/unix/netrc.php).

It is possible to set the login information on connect without using a netrc file.

In [3]:
xnat.connect?

The session is the main entry point for the module. It exposes part of the archive as objects.

In [4]:
sandbox = session.projects['nosetests']
print(sandbox)

<ProjectData nosetests>


In [5]:
sandbox.description

'Random_project_description_89'

In [6]:
new_description = 'Random_project_description_{}'.format(random.randint(0, 100))
print('Changing description to: {}'.format(new_description))
sandbox.description = new_description

Changing description to: Random_project_description_36


In [7]:
sandbox.description

'Random_project_description_36'

In [8]:
# Get a list of the subjects
sandbox.subjects

<XNATListing (CENTRAL_S04325, 5f45e50ffe5311e49e3fa8206633b88a): <SubjectData CENTRAL_S04325>, (CENTRAL_S04322, 8fc8b382fe4b11e49807a8206633b88a): <SubjectData CENTRAL_S04322>, (CENTRAL_S04328, a48e25c7fe5411e4bc06a8206633b88a): <SubjectData CENTRAL_S04328>, (CENTRAL_S01861, hobbit): <SubjectData CENTRAL_S01861>, (CENTRAL_S04346, 0bd89ea3ff2011e4a9d6a8206633b88a): <SubjectData CENTRAL_S04346>, (CENTRAL_S04361, 91738c8cff2211e496cfa8206633b88a): <SubjectData CENTRAL_S04361>, (515502d2da9111df848d001c2304e1df, 515502d2da9111df848d001c2304e1df): <SubjectData 515502d2da9111df848d001c2304e1df>, (CENTRAL_S04364, ade2a647ff2211e4bbc3a8206633b88a): <SubjectData CENTRAL_S04364>, (CENTRAL_S04343, 8b20cf97fe5911e48cf7a8206633b88a): <SubjectData CENTRAL_S04343>, (CENTRAL_S04340, 947d6466fe5711e4b11fa8206633b88a): <SubjectData CENTRAL_S04340>, (CENTRAL_S04367, 103626e3ff2311e4b620a8206633b88a): <SubjectData CENTRAL_S04367>, (NOSETESTS_001, label_001): <SubjectData NOSETESTS_001>, (CENTRAL_S04349, a

Note that the entries are in the form `(CENTRAL_S01824, custom_label): <SubjectData CENTRAL_S01824>`. This does not mean that the key is `(CENTRAL_S01824, custom_label)`, but that both the keys `CENTRAL_S01824` and `custom_label` can be used for lookup. The first key is always the XNAT internal id, the second key is defined as:
* project: the name
* subject: the label
* experiment: the label
* scan: the scantype
* resource: label
* file: filename

In [9]:
subject = sandbox.subjects['bla_subject']

In [10]:
print('Before change:')
print('Gender: {}'.format(subject.demographics.gender))
print('Initials: {}'.format(subject.initials))

# Change gender and initials. Flip them between male and female and JC and PI
subject.demographics.gender = 'female' if subject.demographics.gender == 'male' else 'male'
subject.initials = 'JC' if subject.initials == 'PI' else 'PI'

print('After change:')
print('Gender: {}'.format(subject.demographics.gender))
print('Initials: {}'.format(subject.initials))

Before change:
Gender: female
Initials: PI
After change:
Gender: male
Initials: JC


There is some basic value checking before assignment are carried out. It uses the xsd directives when available. For example:

In [11]:
subject.demographics.gender = 'martian'

ValueError: gender has to be one of: "male", "female", "other", "unknown", "M", "F"

## Custom Variables

In xnatpy custom variables are exposed as a simple mapping type that is very similar to a dictionary.

In [12]:
print(subject.fields)

<VariableMap {u'test_field': u'1337'}>


In [13]:
# Add something
subject.fields['test_field'] = 42
print(subject.fields)

<VariableMap {u'test_field': u'42'}>


In [14]:
subject.fields['test_field']

u'42'

Note that custom variables are always stored as string in the database. So the value is always casted to a string. Also the length of a value is limited because they are passed on the requested url.

The custom variables are by default not visible in the UI, there are special settings in the UI to make them appear. Defined variables in the UI that are not set, are just not appearing in the fields dictionary.

To avoid `KeyError`, it would be best to use `subject.fields.get('field_name')` which returns None when not available.

### Downloading stuff

Downloading with xnatpy is fairly simple. Most objects that are downloadable have a `.download` method. There is also a `download_dir` method that downloads the zip and unpacks it to the target directory.

In [15]:
download_dir = os.path.expanduser('~/xnatpy_temp')
print('Using {} as download directory'.format(download_dir))
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
sandbox.subjects['515502d2da9111df848d001c2304e1df'].download_dir(download_dir)

Using /home/hachterberg/xnatpy_temp as download directory
Downloaded subject to /home/hachterberg/xnatpy_temp/515502d2da9111df848d001c2304e1df


### Close the session!

In [16]:
# Don't forget to disconnect to close cleanly and clean up temporary things!
session.disconnect()

## Context operator

It is also possible to use xnatpy in a context, which guarantees clean closure of the connections etc.

In [ ]:
with xnat.connect('https://central.xnat.org', user='nosetests', password='nose2016') as session:
    print('Nosetests project description: {}'.format(session.projects['nosetests'].description))
    
# Here the session will be closed properly, even if there were exceptions within the `with` context

## Ideas for the future

Currently I am thinking on two different additions and how to implement that best.
* Creation of new objects
* XNAT searches

This illustrates my current ideas, but these are not implemented yet. If you have an opinion about this, let me know!

#### Object creation

or alternatively

#### Searches

The idea is to create something similar to SQLAlchemy